# Data-driven Darcy Flow Using FNO and its Variants

In this notebook we will cover some theory behind the Fourier Neural Operators and some of their variants. We will then apply these architectures on a Darcy flow field prediction problem in a data-driven fashion. 

### Learning Outcomes
1. A brief theory behind the Fourier Neural Operators (FNO), Adaptive Fourier Neural Operators (AFNO) and Physics-informed Neural Operators (PINO) 
2. How to set up and train the three models in Modulus
3. Differences between the three variants

## Fourier Neural Operators (FNO)

Fourier neural operator (FNO) is a data-driven architecture which can be used to parameterize solutions for a distribution of PDE solutions. The key feature of FNO is the spectral convolutions: operations that place the integral kernel in Fourier space. The spectral convolution (Fourier integral operator) is defined as follows:

\begin{equation}
(\mathcal{K}(\mathbf{w})\phi)(x) = \mathcal{F}^{-1}(R_{\mathbf{W}}\cdot \left(\mathcal{F}\right)\phi)(x), \quad \forall x \in D
\end{equation}

where $\mathcal{F}$ and $\mathcal{F}^{-1}$ are the forward and inverse Fourier transforms, respectively.
$R_{\mathbf{w}}$ is the transformation which contains the learn-able parameters $\mathbf{w}$. Note this operator is calculated
over the entire *structured Euclidean* domain $D$ discretized with $n$ points.

Fast Fourier Transform (FFT) is used to perform the Fourier transforms efficiently and the resulting transformation $R_{\mathbf{w}}$ is just a finite size matrix of learn-able weights. Inside the spectral convolution, the Fourier coefficients are truncated to only the lower modes which allows explicit control over the dimensionality of the spectral space and linear operator.

The FNO model is the composition of a fully-connected "lifting" layer, $L$ spectral convolutions with point-wise linear skip connections and a decoding point-wise fully-connected neural network at the end.

\begin{equation}
u_{net}(\Phi;\theta) = \mathcal{Q}\circ \sigma(W_{L} + \mathcal{K}_{L}) \circ ... \circ \sigma(W_{1} + \mathcal{K}_{1})\circ \mathcal{P}(\Phi), \quad \Phi=\left\{\phi(x); \forall x \in D\right\}
\end{equation}

...in which $\sigma(W_{i} + \mathcal{K}_{i})$ is the spectral convolution layer $i$ with the point-wise linear transform $W_{i}$ and activation function $\sigma(\cdot)$. $\mathcal{P}$ is the point-wise lifting network that projects the input into a higher-dimensional latent space, $\mathcal{P}: \mathbb{R}^{d_in} \rightarrow \mathbb{R}^{k}$.

Similarly, $\mathcal{Q}$ is the point-wise fully-connected decoding network, $\mathcal{P}: \mathbb{R}^{k} \rightarrow \mathbb{R}^{d_out}$. Since all fully-connected components of FNO are point-wise operations, the model is invariant to the dimensionality of the input.

<img src="fno_darcy.png" alt="Drawing" style="width: 900px;"/>

For more details, please refer the [Modulus User Documentation](https://docs.nvidia.com/deeplearning/modulus/user_guide/theory/architectures.html#fourier-neural-operator).

## Adaptive Fourier Neural Operators (AFNO)

In contrast with the Fourier Neural Operator which has a convolutional architecture, the AFNO leverages contemporary transformer architectures in the computer vision domain. Vision transformers have delivered tremendous success in computer vision. This is primarily due to effective self-attention mechanisms. To cope with this challenge, Guibas et al. proposed [Adaptive Fourier Neural Operator (AFNO)](https://www.researchgate.net/publication/356601975_Adaptive_Fourier_Neural_Operators_Efficient_Token_Mixers_for_Transformers) as an efficient attention mechanism in the Fourier Domain. AFNO is based on principled foundation of operator learning which allows us to frame attention as a continuous global convolution efficiently in the Fourier domain. To handle challenges in vision such as discontinuities in images and high-resolution inputs, AFNO proposes principled architectural modifications to FNO which results in memory and computational efficiency. This includes imposing a block-diagonal structure on the channel mixing weights, adaptively sharing weights across tokens, and sparsifying the frequency modes via soft-thresholding and shrinkage. 

The AFNO model typically includes the following steps:
1. Dividing the input image into a regular grid with $h \times w$ equal sized patches of size $p\times p$.
2. Embed the patch into a token of size $d$, the embedding dimension resulting in a token tensor ($X_{h\times w \times d}$) of size $h \times w \times d$. 
3. Pass the token through multiple layers of transformer architecture performing spatial and channel mixing. 
4. At the end of all the transformer layers, convert the feature tensor back to image space using a linear decoder.

For each layer in the Step 3, the AFNO architecture implements the following operations: 

The token tensor is first transformed to the Fourier domain with

\begin{equation}
z_{m,n} = [\mathrm{DFT}(X)]_{m,n},
\end{equation}

where $m,n$ is the index the patch location and DFT denotes a 2D discrete Fourier transform. The model then applies token weighting in the Fourier domain and promotes sparsity with a Soft-Thresholding and Shrinkage operation as

\begin{equation} 
\tilde{z}_{m,n} = S_{\lambda} ( \mathrm{MLP}(z_{m,n})),
\end{equation}

where $S_{\lambda}(x) = \mathrm{sign}(x) \max(|x| - \lambda, 0)$ with the sparsity controlling parameter $\lambda$, and $\mathrm{MLP(\cdot)}$ is a 2-layer multi-layer perceptron with block-diagonal weight matrices which are shared across all patches. 

The last operation in an ANFO layer is an inverse Fourier to transform back to the patch domain and add a residual connection as

\begin{equation}
y_{m,n} = [\mathrm{IDFT}(\tilde{Z})]_{m,n} + X_{m,n}.
\end{equation}

<img src="afno_darcy.png" alt="Drawing" style="width: 900px;"/>

The complete mathematical description of the AFNO model is beyond the scope of this material, and we refer you to [Modulus User Documentation](https://docs.nvidia.com/deeplearning/modulus/user_guide/theory/architectures.html#adaptive-fourier-neural-operator) for additional details.

## Physics-informed Neural Operators

The Physics-Informed Neural Operator (PINO) was introduced by Li et al. in this [paper](https://arxiv.org/abs/2111.03794). The PINO approach for surrogate modeling of PDE systems effectively combines the data-informed supervised learning framework of the FNO with the physics-informed learning framework. The PINO incorporates a PDE loss $\mathcal{L}_{pde}$ to the Fourier Neural Operator. This reduces the amount of data required to train a surrogate model, since the PDE loss constrains the solution space. 
The PDE loss also enforces physical constraints on the solution computed by a surrogate ML model, making it an attractive option as a verifiable, accurate and interpretable ML surrogate modeling tool.

To explain the key concepts behind PINO, consider a stationary PDE system. Following the notation used in the [paper](https://arxiv.org/abs/2111.03794), we consider a PDE represented by,

\begin{equation} 
\mathcal{P}(u, a) = 0 , \text{ in } D \subset \mathbb{R}^d,
\end{equation}

\begin{equation}
u = g ,  \text{ in } \partial D.
\end{equation}

Here, $\mathcal{P}$ is a Partial Differential Operator, $a$ are the coefficients/parameters and $u$ is the PDE solution.

In the FNO framework, the surrogate ML model is given by the solution operator $\mathcal{G}^\dagger_{\theta}$, which maps any given coefficient in the coefficient space $a$ to the solution $u$. The FNO is trained in a supervised fashion using training data in the form of input/output pairs $\lbrace a_j, u_j \rbrace_{j = 1}^N$. The training loss for the FNO is given by summing the data loss, $\mathcal{L}_{data}(\mathcal{G}_\theta) = \lVert u - \mathcal{G}_\theta(a)  \rVert^2$ over all training pairs $\lbrace a_i, u_i,  \rbrace_{i=1}^N$.

In the PINO framework, the solution operator is optimized with an additional PDE loss given by $\mathcal{L}_{pde}(a, \mathcal{G}_{\theta}(a))$ computed over i.i.d. samples $a_j$ from an appropriate supported distribution in parameter/coefficient space.

In general, the PDE loss involves computing the PDE operator which in turn involves computing the partial derivatives of the Fourier Neural Operator ansatz. In general, this is nontrivial. The key set of innovations in the PINO are the various ways to compute the partial derivatives of the operator ansatz, viz:

1. Numerical differentiation using a Finite-Difference Method (FDM).
2. Numerical differentiation computed via spectral derivative. 
3. Hybrid differentiation based on a combination of first-order "exact" derivatives and second-order FDM derivatives. 

For more details, please refer the [Modulus User Documentation](https://docs.nvidia.com/deeplearning/modulus/user_guide/theory/architectures.html#physics-informed-neural-operator).

Now, with this theoretical background, let's solve the Darcy flow problem using these three approaches.

## Problem Description

The Darcy PDE is a second order, elliptic PDE with the following form:

\begin{equation}
-\nabla \cdot \left(k(\textbf{x})\nabla u(\textbf{x})\right) = f(\textbf{x}), \quad \textbf{x} \in D,
\end{equation}

...in which $u(\textbf{x})$ is the flow pressure, $k(\textbf{x})$ is the permeability field and $f(\cdot)$ is the
forcing function. The Darcy flow can parameterize a variety of systems including flow through porous media, elastic materials 
and heat conduction. Here, we will define the domain as a 2D unit square  $D=\left\{x,y \in (0,1)\right\}$ with the boundary condition $u(\textbf{x})=0, \textbf{x}\in\partial D$. Both the permeability and flow fields are discretized into a 2D matrix $\textbf{K}, \textbf{U} \in \mathbb{R}^{N \times N}$.

The goal of this problem is to develop a surrogate model that learns the mapping between a permeability field and the pressure field,
$\textbf{K} \rightarrow \textbf{U}$, for a distribution of permeability fields $\textbf{K} \sim p(\textbf{K})$.
In this problem, you are *not* learning just a single solution but rather a distribution.

## Download the Required Data

Before starting any coding, we need to make sure that we have both the training and the validation data. The training and validation datasets for this example can be found on the [Fourier Neural Operator GitHub page](https://github.com/zongyi-li/fourier_neural_operator). The script [`utilities.py`](../../source_code/darcy/utilities.py) is an automated script for downloading and converting this dataset. This requires the package [gdown](https://github.com/wkentaro/gdown) which can be easily installed through `pip install gdown`.

In [ ]:
!pip install gdown
import os
os.chdir('../../source_code/darcy/')

from utilities import download_FNO_dataset, load_FNO_dataset
from modulus.dataset import HDF5GridDataset
from modulus.key import Key

download_FNO_dataset("Darcy_241", outdir="datasets/")

## Part 1: Solution Using FNO

Let's see how to solve the problem using the FNOs. The complete problem setup can be found in the [`darcy_FNO_lazy.py`](../../source_code/darcy/darcy_FNO_lazy.py) script, but here for the sake of understanding, we will build the constraints and the case step by step. [`config_FNO.yaml`](../../source_code/darcy/conf/config_FNO.yaml) lists the configs required for the problem. Let's start by loading them:

In [ ]:
from modulus.hydra import to_yaml, to_absolute_path
from modulus.hydra.utils import compose
from modulus.hydra.config import ModulusConfig

cfg = compose(config_path="../../source_code/darcy/conf", config_name="config_FNO")
cfg.network_dir = 'outputs/darcy_FNO'
print(to_yaml(cfg))

### Loading the Data

For this data-driven problem, the first step is to get the training data into Modulus. Prior to loading data, we can set any normalization value that we want to apply to the data. For this dataset, we calculated the scale and shift parameters for both the input permeability field and output pressure. Then, we set this normalization inside Modulus by providing the scale/shift to each key, `Key(name, scale=(shift, scale))`.

In [ ]:
input_keys = [Key("coeff", scale=(7.48360e00, 4.49996e00))]
output_keys = [Key("sol", scale=(5.74634e-03, 3.88433e-03))]

There are two approaches for loading data. The first is to use eager loading where you immediately read the entire dataset into memory at one time. Alternatively, you can use lazy loading where the data is loaded on a per-example basis as the model needs it for training. The eager loading eliminates potential overhead from reading data from disc during training, however this cannot scale to large datasets. Lazy loading is used in this example for both the training and test datasets to demonstrate this utility for larger problems.

In [ ]:
train_path = to_absolute_path("datasets/Darcy_241/piececonst_r241_N1024_smooth1.hdf5")
test_path = to_absolute_path("datasets/Darcy_241/piececonst_r241_N1024_smooth2.hdf5")

# make datasets
train_dataset = HDF5GridDataset(train_path, invar_keys=["coeff"], outvar_keys=["sol"], n_examples=1000)
test_dataset = HDF5GridDataset(test_path, invar_keys=["coeff"], outvar_keys=["sol"], n_examples=100)

If you are interested in seeing how the eager loading works, please refer [`darcy_FNO.py`](../../source_code/darcy/darcy_FNO.py) script

### Initializing the Model and Domain, and Adding Constraints

Initializing the model and domain follows the same steps as the other PINN models we saw earlier. 

For the physics-informed problems in Modulus, we typically need to define a geometry and constraints based on boundary conditions and governing equations. Here, the only constraint is a [`SupervisedGridConstraint`](https://docs.nvidia.com/deeplearning/modulus/api/modulus.domain.constraint.html#modulus.domain.constraint.discrete.SupervisedGridConstraint) which performs standard supervised training on grid data. This constraint supports the use of multiple workers, which are particularly important when using lazy loading. Let's see how this can be achieved in the code:

In [ ]:
from modulus.hydra import instantiate_arch
from modulus.domain import Domain
from modulus.domain.constraint import SupervisedGridConstraint

# make list of nodes to unroll graph on
model = instantiate_arch(
    input_keys=input_keys,
    output_keys=output_keys,
    cfg=cfg.arch.fno,
)
nodes = model.make_nodes(name="FNO", jit=cfg.jit)

# make domain
domain = Domain()

# add constraints to domain
supervised = SupervisedGridConstraint(
    nodes=nodes,
    dataset=train_dataset,
    batch_size=cfg.batch_size.grid,
    num_workers=4,  # number of parallel data loaders
)
domain.add_constraint(supervised, "supervised")

**Note:** Grid data refers to data that can be defined in a tensor like an image. Inside Modulus, this grid of data typically represents a spatial domain and should follow the standard dimensionality of `[batch, channel, xdim, ydim, zdim]` where channel is the dimensionality of your state variables. Both Fourier and convolutional models use grid-based data to efficiently learn and predict entire domains in one forward pass, which contrasts to the point-wise predictions of standard PINN approaches.

### Adding Data Validator

We can then add the validation data to the domain using [`GridValidator`](https://docs.nvidia.com/deeplearning/modulus/api/modulus.domain.validator.html#modulus.domain.validator.discrete.GridValidator) which should be used when dealing with structured data. 

In [ ]:
from modulus.domain.validator import GridValidator
from modulus.utils.io.plotter import GridValidatorPlotter

# add validator
val = GridValidator(
    nodes,
    dataset=test_dataset,
    batch_size=cfg.batch_size.validation,
    plotter=GridValidatorPlotter(n_examples=5),
)
domain.add_validator(val, "test")

### Solver and Training the Model 

We can create a solver by using the domain we just created along with the other configurations that define the optimizer choices, settings using Modulus’ `Solver` class. The solver can then be executed using the `solve` method.

In [ ]:
# to make the logging work in the jupyter cells
# Need to execute this only once!
import logging
logging.getLogger().addHandler(logging.StreamHandler())

In [ ]:
import os
from modulus.solver import Solver

# optional set appropriate GPU in case of multi-GPU machine
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]="2"

# make solver
slv = Solver(cfg, domain)

# start solver
slv.solve()

### Results and Post-processing

The checkpoint directory is saved based on the results recording frequency specified in the `rec_results_freq` parameter of its derivatives. The network directory folder contains several plots of different validation predictions. Several are shown below, and you can see that the model is able to accurately predict the pressure field for permeability fields it had not seen previously. These visualizations can also be viewed during the training using the Tensorboard. 

<figure>
    <figcaption>FNO validation predictions. (Left to right) Input permeability, true pressure, predicted pressure, error.</figcaption>
    <img src="fno_darcy_pred1.png" alt="Drawing" style="width: 900px;"/>
    <img src="fno_darcy_pred2.png" alt="Drawing" style="width: 900px;"/>
    <img src="fno_darcy_pred3.png" alt="Drawing" style="width: 900px;"/>
</figure>

## Part 2: Solution Using AFNO

Let's see how to solve the problem using the AFNOs. The complete problem setup can be found in the [`darcy_AFNO.py`](../../source_code/darcy/darcy_AFNO.py) script, but here for the sake of understanding, we will build the constraints and the case step by step. [`config_AFNO.yaml`](../../source_code/darcy/conf/config_AFNO.yaml) lists the configs required for the problem. Let's start by loading them. The process of setting the AFNO training is very similar from a case setup standpoint hence we will only highlight the important differences wherever necessary. 

The AFNO is based on the ViT transformer architecture and requires tokenization of the inputs. Here each token is a patch of the image with a patch size defined in the configuration file through the parameter `patch_size`. The `embed_dim` parameter defines the size of the latent embedded features used inside the model for each patch. 

In [ ]:
cfg = compose(config_path="../../source_code/darcy/conf", config_name="config_AFNO")
cfg.network_dir = 'outputs/darcy_AFNO'
print(to_yaml(cfg))

### Loading the Data

Loading both the training and validation datasets into memory follows a similar process as seen in the FNO. The inputs for AFNO need to be perfectly divisible by the specified patch size (in this case, `patch_size=16`), which is not the case for this dataset. Therefore, we trim the input/output features such that they have appropriate dimensionality `241x241 -> 240x240`.

In [ ]:
from modulus.dataset import DictGridDataset

# load training/ test data
input_keys = [Key("coeff", scale=(7.48360e00, 4.49996e00))]
output_keys = [Key("sol", scale=(5.74634e-03, 3.88433e-03))]

invar_train, outvar_train = load_FNO_dataset(
    "datasets/Darcy_241/piececonst_r241_N1024_smooth1.hdf5",
    [k.name for k in input_keys],
    [k.name for k in output_keys],
    n_examples=1000,
)
invar_test, outvar_test = load_FNO_dataset(
    "datasets/Darcy_241/piececonst_r241_N1024_smooth2.hdf5",
    [k.name for k in input_keys],
    [k.name for k in output_keys],
    n_examples=100,
)

# get training image shape
img_shape = [
    next(iter(invar_train.values())).shape[-2],
    next(iter(invar_train.values())).shape[-1],
]

# crop out some pixels so that img_shape is divisible by patch_size of AFNO
img_shape = [s - s % cfg.arch.afno.patch_size for s in img_shape]
print(f"cropped img_shape: {img_shape}")
for d in (invar_train, outvar_train, invar_test, outvar_test):
    for k in d:
        d[k] = d[k][:, :, : img_shape[0], : img_shape[1]]
        print(f"{k}: {d[k].shape}")

# make datasets
train_dataset = DictGridDataset(invar_train, outvar_train)
test_dataset = DictGridDataset(invar_test, outvar_test)

### Initializing the Model and Domain, Adding Constraints and Validators.

These steps are the same as what we saw in the case of FNOs. For AFNO, we calculate the size of the domain after loading the dataset. The domain needs to be defined in the AFNO model, which is provided with the inclusion of the keyword argument `img_shape` in the `instantiate_arch` call. 

In [ ]:
# make list of nodes to unroll graph on
model = instantiate_arch(
    input_keys=input_keys,
    output_keys=output_keys,
    cfg=cfg.arch.afno,
    img_shape=img_shape,
)
nodes = [model.make_node(name="AFNO", jit=cfg.jit)]

# make domain
domain = Domain()

# add constraints to domain
supervised = SupervisedGridConstraint(
    nodes=nodes,
    dataset=train_dataset,
    batch_size=cfg.batch_size.grid,
)
domain.add_constraint(supervised, "supervised")

# add validator
val = GridValidator(
    nodes,
    dataset=test_dataset,
    batch_size=cfg.batch_size.validation,
    plotter=GridValidatorPlotter(n_examples=5),
)
domain.add_validator(val, "test")

### Solver and Training the Model

Once the domain and the configuration is set up, the `Solver` can be defined, and the training can be started as seen before.

In [ ]:
# make solver
slv = Solver(cfg, domain)

# start solver
slv.solve()

### Results and Post-processing

The checkpoint directory is saved based on the results recording frequency specified in the `rec_results_freq` parameter of its derivatives. The network directory folder contains several plots of the different validation predictions, some of which are shown below. 

<figure>
    <figcaption>AFNO validation predictions. (Left to right) Input permeability, true pressure, predicted pressure, error.</figcaption>
    <img src="afno_darcy_pred1.png" alt="Drawing" style="width: 900px;"/>
    <img src="afno_darcy_pred2.png" alt="Drawing" style="width: 900px;"/>
    <img src="afno_darcy_pred3.png" alt="Drawing" style="width: 900px;"/>
</figure>

It is important to recognize that AFNO's strength lies in its ability to scale to a much larger model size and datasets than what is used in this notebook/example. While not illustrated here, this example demonstrates the fundamental implementation of data-driven training using the AFNO architecture in Modulus for you to extend to larger problems.

## Part 3: Solution Using PINO

The key difference between PINO and FNO is that PINO adds a physics-informed term to the loss function of FNO. As discussed further in the theory, the PINO loss function is described by:

\begin{equation}
\mathcal{L} = \mathcal{L}_{data} + \mathcal{L}_{pde},
\end{equation}

where,
\begin{equation}
\mathcal{L}_{data} = \lVert u - \mathcal{G}_\theta(a)  \rVert^2 ,
\end{equation}

...where $\mathcal{G}_\theta(a)$ is an FNO model with learnable parameters $\theta$ and input field $a$, and 
$\mathcal{L}_{pde}$ is an appropriate PDE loss. For the 2D Darcy problem this is given by:

\begin{equation}
\mathcal{L}_{pde} = \lVert -\nabla \cdot \left(k(\textbf{x})\nabla \mathcal{G}_\theta(a)(\textbf{x})\right) - f(\textbf{x}) \rVert^2 ,
\end{equation}

…where $k(\textbf{x})$ is the permeability field, $f(\textbf{x})$ is the forcing function equal to 1 in this case, and $a=k$ in this case.

Note that the PDE loss involves computing various partial derivatives of the FNO ansatz, $\mathcal{G}_\theta(a)$. 
In Modulus, three different methods for computing these are provided. These are based on the original PINO paper:

1. Numerical differentiation computed via Finite-Difference Method (FDM)
2. Numerical differentiation computed via spectral derivative
3. Hybrid differentiation based on a combination of first-order "exact" derivatives and second-order FDM derivatives

The first 2 approaches are the same as proposed in the original paper. The third approach is a modification of the "exact" approach proposed in the paper.
This method is slower and more memory intensive than the numerical derivative approaches when computing second order derivatives
because it requires the computation of a Hessian matrix. 
Instead, a "hybrid" approach is provided which offers a compromise by combining first-order "exact"(the exact method is not technically exact because it uses a combination of numerical spectral derivatives and exact differentiation, see original paper for more details) derivatives and second-order FDM derivatives.

Now considering the problem setup, it is largely the same as the FNO example, except that the PDE loss is defined and the FNO model is constrained using it. This process is described in detail when we define the PDE loss. 


Let's start by loading the configs for this problem. The configuration for this problem is similar to the FNO example, but importantly there is an extra parameter `custom.gradient_method` where the method for computing the gradients in the PDE loss is selected. This can be one of `fdm`, `fourier`, or `hybrid` corresponding to the three options above. The balance between the data and PDE terms in the loss function can also be controlled using the `loss.weights` parameter group. The [`config_PINO.yaml`](../../source_code/darcy/conf/config_PINO.yaml) is loaded below.

In [ ]:
cfg = compose(config_path="../../source_code/darcy/conf", config_name="config_PINO")
cfg.network_dir = 'outputs/darcy_PINO'
print(to_yaml(cfg))

### Define PDE Loss for Grid Data

For this example, a custom PDE residual calculation is defined using the various approaches proposed above. The process of defining a custom PDE residual using SymPy and auto-diff was discussed in the [notebooks on PINNs](../diffusion_1d/Diffusion_Problem_Notebook.ipynb). For this problem, we will not be relying on standard auto-diff for calculating the derivatives, instead we want to explicitly define how the residual is calculated using a custom `torch.nn.Module` called `Darcy`. The purpose of this module is to compute and return the Darcy PDE residual given the input and output tensors of the FNO model, which is done via its `.forward(...)` method.

In [ ]:
import torch
import torch.nn.functional as F
from typing import Dict

# import the custom ops
from ops import dx, ddx

class Darcy(torch.nn.Module):
    "Custom Darcy PDE definition for PINO"

    def __init__(self, gradient_method: str = "hybrid"):
        super().__init__()
        self.gradient_method = str(gradient_method)

    def forward(self, input_var: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
        # get inputs
        u = input_var["sol"]
        c = input_var["coeff"]
        dcdx = input_var["Kcoeff_y"]  # data is reversed
        dcdy = input_var["Kcoeff_x"]

        dxf = 1.0 / u.shape[-2]
        dyf = 1.0 / u.shape[-1]
        # Compute gradients based on method
        # Exact first order and FDM second order
        if self.gradient_method == "hybrid":
            dudx_exact = input_var["sol__x"]
            dudy_exact = input_var["sol__y"]
            dduddx_fdm = ddx(
                u, dx=dxf, channel=0, dim=0, order=1, padding="replication"
            )
            dduddy_fdm = ddx(
                u, dx=dyf, channel=0, dim=1, order=1, padding="replication"
            )
            # compute darcy equation
            darcy = (
                1.0
                + (dcdx * dudx_exact)
                + (c * dduddx_fdm)
                + (dcdy * dudy_exact)
                + (c * dduddy_fdm)
            )
        # FDM gradients
        elif self.gradient_method == "fdm":
            dudx_fdm = dx(u, dx=dxf, channel=0, dim=0, order=1, padding="replication")
            dudy_fdm = dx(u, dx=dyf, channel=0, dim=1, order=1, padding="replication")
            dduddx_fdm = ddx(
                u, dx=dxf, channel=0, dim=0, order=1, padding="replication"
            )
            dduddy_fdm = ddx(
                u, dx=dyf, channel=0, dim=1, order=1, padding="replication"
            )
            # compute darcy equation
            darcy = (
                1.0
                + (dcdx * dudx_fdm)
                + (c * dduddx_fdm)
                + (dcdy * dudy_fdm)
                + (c * dduddy_fdm)
            )
        # Fourier derivative
        elif self.gradient_method == "fourier":
            dim_u_x = u.shape[2]
            dim_u_y = u.shape[3]
            u = F.pad(
                u, (0, dim_u_y - 1, 0, dim_u_x - 1), mode="reflect"
            )  # Constant seems to give best results
            f_du, f_ddu = fourier_derivatives(u, [2.0, 2.0])
            dudx_fourier = f_du[:, 0:1, :dim_u_x, :dim_u_y]
            dudy_fourier = f_du[:, 1:2, :dim_u_x, :dim_u_y]
            dduddx_fourier = f_ddu[:, 0:1, :dim_u_x, :dim_u_y]
            dduddy_fourier = f_ddu[:, 1:2, :dim_u_x, :dim_u_y]
            # compute darcy equation
            darcy = (
                1.0
                + (dcdx * dudx_fourier)
                + (c * dduddx_fourier)
                + (dcdy * dudy_fourier)
                + (c * dduddy_fourier)
            )
        else:
            raise ValueError(f"Derivative method {self.gradient_method} not supported.")

        # Zero outer boundary
        darcy = F.pad(darcy[:, :, 2:-2, 2:-2], [2, 2, 2, 2], "constant", 0)
        # Return darcy
        output_var = {
            "darcy": dxf * darcy,
        }  # weight boundary loss higher
        return output_var

The gradients of the FNO solution are computed according to the gradient method selected above. The FNO model automatically outputs first order gradients when the `hybrid` method is used, and so no extra computation of these is necessary. Furthermore, note that the gradients of the permeability field are already included as tensors in the FNO input training data (with keys `Kcoeff_x` and `Kcoeff_y`) and so these do not need to be computed.

### Load the Data and Initialize the Model 

The data loading follows a similar process as the FNO example. Only for the case where hybrid gradient method is used, you need to additionally instruct the model to output the appropriate gradients by specifying these gradients in its output keys.

We will incorporate the `Darcy` model we defined earlier into Modulus by wrapping it into a Modulus `Node`. This ensures the module is incorporated into Modulus’ computational graph and can be used to optimize the FNO.

In [ ]:
import numpy as np
from modulus.dataset import DictGridDataset
from modulus.node import Node

# load training/ test data
input_keys = [
    Key("coeff", scale=(7.48360e00, 4.49996e00)),
    Key("Kcoeff_x"),
    Key("Kcoeff_y"),
]
output_keys = [
    Key("sol", scale=(5.74634e-03, 3.88433e-03)),
]

invar_train, outvar_train = load_FNO_dataset(
    "datasets/Darcy_241/piececonst_r241_N1024_smooth1.hdf5",
    [k.name for k in input_keys],
    [k.name for k in output_keys],
    n_examples=cfg.custom.ntrain,
)
invar_test, outvar_test = load_FNO_dataset(
    "datasets/Darcy_241/piececonst_r241_N1024_smooth2.hdf5",
    [k.name for k in input_keys],
    [k.name for k in output_keys],
    n_examples=cfg.custom.ntest,
)

# Define FNO model
if cfg.custom.gradient_method == "hybrid":
    output_keys += [
        Key("sol", derivatives=[Key("x")]),
        Key("sol", derivatives=[Key("y")]),
    ]
model = instantiate_arch(
    input_keys=[input_keys[0]],
    output_keys=output_keys,
    cfg=cfg.arch.fno,
    domain_length=[1.0, 1.0],
)

# Make custom Darcy residual node for PINO
inputs = [
    "sol",
    "coeff",
    "Kcoeff_x",
    "Kcoeff_y",
]
if cfg.custom.gradient_method == "hybrid":
    inputs += [
        "sol__x",
        "sol__y",
    ]
darcy_node = Node(
    inputs=inputs,
    outputs=["darcy"],
    evaluate=Darcy(gradient_method=cfg.custom.gradient_method),
    name="Darcy Node",
)
nodes = model.make_nodes(name="FNO", jit=False) + [darcy_node]

### Initializing the Domain, Adding Constraints and Validators

Finally, we can add constraints to our model similarly to the FNO example. The same `SupervisedGridConstraint` can be used to include the PDE loss term we need to define additional target values for the `darcy` output variable defined above (zeros, to minimize the PDE residual) and add them to the `outvar_train` dictionary.

In [ ]:
# make domain
domain = Domain()

# add additional constraining values for darcy variable
outvar_train["darcy"] = np.zeros_like(outvar_train["sol"])

train_dataset = DictGridDataset(invar_train, outvar_train)
test_dataset = DictGridDataset(invar_test, outvar_test)

# add constraints to domain
supervised = SupervisedGridConstraint(
    nodes=nodes,
    dataset=train_dataset,
    batch_size=cfg.batch_size.grid,
)
domain.add_constraint(supervised, "supervised")

# add validator
val = GridValidator(
    nodes,
    dataset=test_dataset,
    batch_size=cfg.batch_size.validation,
    plotter=GridValidatorPlotter(n_examples=5),
    requires_grad=True,
)
domain.add_validator(val, "test")

### Solver and Training the Model

Once the domain and the configuration is set up, the `Solver` can be defined, and the training can be started as seen before.

In [ ]:
# make solver
slv = Solver(cfg, domain)

# start solver
slv.solve()

### Results and Post-processing

The network directory folder contains several plots of different validation predictions. One of them is shown below.

<figure>
    <figcaption>PINO validation predictions. (Left to right) Input permeability and its spatial derivatives, true pressure, predicted pressure, error.</figcaption>
    <img src="pino_darcy_pred.png" alt="Drawing" style="width: 900px;"/>
</figure>

### Comparison to FNO

The Tensorboard plot below compares the validation loss of PINO (all three gradient methods) and FNO. You can see that with large amounts of training data (1000 training examples), both FNO and PINO perform similarly. 

<img src="pino_darcy_tensorboard1.png" alt="Drawing" style="width: 600px;"/>

A benefit of PINO is that the PDE loss regularizes the model, meaning that it can be more efficient in "small data" regimes. The plot below shows the validation loss when both models are trained with only 100 training examples. In this case, we find that PINO outperforms FNO. 

<img src="pino_darcy_tensorboard2.png" alt="Drawing" style="width: 600px;"/>

## Next

In the final section of this course you will perform a small exercise to solve a fluid mechanics problem involving the solution to the Navier-Stokes equations by using the PINN approach.

Please continue to [the next notebook](../chip_2d/Exercise_CFD_Problem_Notebook.ipynb).